<a href="https://colab.research.google.com/github/The237/DeepLearningCourses/blob/main/02_01_Recreer_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# La boîte générique

In [2]:
class Box():
  def __init__(self):
    pass

  def forward(self, inputs):
    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):
    assert derivee_output.shape == self.output.shape, f'La dérivée output reçue a un shape {derivee_output.shape} et différent du shape de output : {self.output.shape}'

    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f'La dérivée input calculée a un shape {self.derivee_input.shape} et différent du shape de input : {self.inputs.shape}'

    return self.derivee_inputs

  def operation(self):
    pass

  def gradient(self,derivee_output):
    pass

 # La boîte paramétrée

In [3]:
class BoxParams():
  def __init__(self, param):
    self.param = param
    pass

  def forward(self, inputs):
    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):

    assert derivee_output.shape == self.output.shape, f'La dérivée output reçue a un shape {derivee_output.shape} et différent du shape de output : {self.output.shape}'

    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f'La dérivée input calculée a un shape {self.derivee_input.shape} et différent du shape de input : {self.inputs.shape}'

    self.derivee_param = self.gradient_param(derivee_output)
    assert self.derivee_param.shape == self.param.shape, f'La dérivée de param a un shape {self.derivee_param.shape} et différent du shape de param : {self.param.shape}'

    return self.derivee_inputs

  def operation(self):
    pass

  def gradient(self,derivee_output):
    pass

  def gradient_param(self,derivee_output):
    pass

# La Classe Dot

In [4]:
class Dot(BoxParams):

  def __init__(self, weights):
    super().__init__(weights)

  def operation(self):
    return np.dot(self.inputs, self.param)

  def gradient(self, derivee_output):
     return np.dot(derivee_output,self.param.T)

  def gradient_param(self, derivee_output):
     return np.dot(self.inputs.T, derivee_output)

  def __repr__(self):
    return "Dot Product"

In [5]:
X = np.array([
    [2,3,-2],
    [4,5,-1],
    [-5,2,3],
    [0,5,4]
])

In [6]:
W = np.array([
    [.4967],
    [-.1383],
    [.6477]
])

In [7]:
np.dot(X,W)

array([[-0.7169],
       [ 0.6476],
       [-0.817 ],
       [ 1.8993]])

In [8]:
M = Dot(weights=W)

In [9]:
out = M.forward(X)
out

array([[-0.7169],
       [ 0.6476],
       [-0.817 ],
       [ 1.8993]])

In [10]:
d_out = np.random.randn(4,1)
d_out

array([[ 1.59522098],
       [-0.56166753],
       [-1.0814617 ],
       [ 1.20234324]])

In [11]:
M.backward(d_out)

array([[ 0.79234626, -0.22061906,  1.03322463],
       [-0.27898026,  0.07767862, -0.36379206],
       [-0.53716203,  0.14956615, -0.70046274],
       [ 0.59720389, -0.16628407,  0.77875772]])

# La Classe Add

In [12]:
class Add(BoxParams):

  def __init__(self, biais):
    super().__init__(biais)

  def operation(self):
    return self.inputs + self.param

  def gradient(self, derivee_output):
     return np.ones_like(self.inputs)*derivee_output

  def gradient_param(self, derivee_output):
      r = np.ones_like(self.param)*derivee_output
      return r.sum(axis = 0).reshape(1, self.param.shape[1])

  def __repr__(self):
    return "Add Biais"

In [13]:
B = np.random.randn(1,1)

In [14]:
B

array([[-0.15864316]])

In [15]:
b = Add(biais = B)
b

Add Biais

In [16]:
out_b = b.forward(out)
out_b

array([[-0.87554316],
       [ 0.48895684],
       [-0.97564316],
       [ 1.74065684]])

In [17]:
d_out

array([[ 1.59522098],
       [-0.56166753],
       [-1.0814617 ],
       [ 1.20234324]])

In [18]:
b.backward(d_out)

array([[ 1.59522098],
       [-0.56166753],
       [-1.0814617 ],
       [ 1.20234324]])

# La Classe Sigmoid

In [19]:
class Sigmoid(Box):
  def __init__(self):
     super().__init__()

  def operation(self):
    return 1 /(1 + np.exp(-1 * self.inputs))

  def gradient(self,derivee_output):
    return self.output * (1 - self.output) * derivee_output

  def __repr__(self):
    return "Sigmoid"

In [20]:
sig = Sigmoid()

In [21]:
sig_out = sig.forward(out_b)
sig_out

array([[0.2941022 ],
       [0.61986066],
       [0.27375713],
       [0.85077048]])

In [22]:
sig.backward(d_out)

array([[ 0.3311776 ],
       [-0.13234764],
       [-0.2150099 ],
       [ 0.15264958]])

# La Classe Loss

In [23]:
class Loss():
  def __init__(self):
    pass

  def forward(self, prediction, target):
    assert prediction.shape == target.shape
    self.prediction = prediction
    self.target = target
    loss = np.mean((self.target -self.prediction)**2)
    return loss

  def backward(self):
    self.loss_derivee = -2 * (self.target - self.prediction)
    assert self.loss_derivee.shape == self.prediction.shape, f'La dérivée loss a un shape {self.loss_derivee.shape} et différent du shape de Prediction : {self.prediction.shape}'

    return self.loss_derivee

In [24]:
Y = np.random.randn(4,1)
Y

array([[ 0.56486772],
       [ 1.48800652],
       [-1.48644887],
       [-0.13822431]])

In [25]:
P = sig_out

In [26]:
mse = Loss()

In [27]:
mse.forward(P, Y)

1.2258567584730986

In [28]:
mse.backward()

array([[-0.54153104],
       [-1.73629171],
       [ 3.52041199],
       [ 1.97798957]])

# La classe Dense

In [29]:
class Dense():

  def __init__(self, neurons, activation = None):
    self.neurons = neurons
    self.activation = activation
    self.params = []
    self.suite = []
    self.initialization = True
    self.derivee_inputs = []

  def build(self, inputs):
    # weights initialization
    np.random.seed(42)
    self.weights = np.random.randn(inputs.shape[1], self.neurons)
    self.biais = np.random.randn(1, self.neurons)
    self.params.append(self.weights)
    self.params.append(self.biais)

    # construction de la suite d'opérations
    self.suite = [Dot(weights= self.params[0]), Add(biais=self.params[1])]
    if self.activation:
      self.suite.append(self.activation)

  def forward(self, inputs):
    if self.initialization:
      self.build(inputs)
      self.initialization = False

      for boite in  self.suite:
        inputs = boite.forward(inputs)

    self.output = inputs

    return self.output

  def backward(self, derivee_output):
    assert derivee_output.shape == self.output.shape

    for box in reversed(self.suite):
      derivee_output = box.backward(derivee_output)
    derivee_inputs = derivee_output

    self.get_layer_gradient()
    return derivee_inputs

  def get_layer_gradient(self):
    self.derivee_params = []

    for box in self.suite:
      if issubclass(box.__class__, BoxParams):
        self.derivee_params.append(box.derivee_param)

  def __repr__(self):
    r = f'Dense Layer(neurons = {self.neurons})'
    if self.activation:
      r +=" avec Sigmoid"
    return r

In [30]:
sigmoid = Sigmoid()
couche = Dense(neurons = 2, activation = sigmoid)
couche

Dense Layer(neurons = 2) avec Sigmoid

In [31]:
X

array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [32]:
couche.forward(X)

array([[0.99320003, 0.99604286],
       [0.99912347, 0.99968533],
       [0.42276305, 0.97817014],
       [0.97978765, 0.99941659]])

In [33]:
couche.params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [34]:
d_out = np.random.randn(4,2)
d_out

array([[-0.46947439,  0.54256004],
       [-0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024],
       [-1.72491783, -0.56228753]])

In [35]:
couche.backward(d_out)

array([[-0.00187061,  0.00120335,  0.00024173],
       [-0.00018133, -0.00048599,  0.00012933],
       [ 0.03497832, -0.02397904, -0.00426045],
       [-0.0169224 , -0.02262434,  0.00807543]])

In [36]:
couche.suite

[Dot Product, Add Biais, Sigmoid]

# La classe Model

In [49]:
class Model():

  def __init__(self, layers):
    self.layers = layers

  def forward(self, inputs):
    for layer in self.layers:
      inputs = layer.forward(inputs)
    self.output = inputs
    return self.output

  def backward(self, loss_derivee):
    assert loss_derivee.shape == self.output.shape
    for layer in reversed(self.layers):
      loss_derivee = layer.backward(loss_derivee)

    return loss_derivee

  def get_params(self):
    self.params = []
    for layer in self.layers:
      self.params.append(layer.params)
    return self.params

  def get_derivee_params(self):
    self.derivee_params = []
    for layer in self.layers:
      self.derivee_params.append(layer.derivee_params)
    return self.derivee_params


  def update(self):
    for param, derivee_param in zip(self.get_params(), self.get_derivee_params()):
      param = param - .01*derivee_param

  def __repr__(self):
    r = "Layers ................."
    for layer in self.layers:
      r+=f" \n {str(layer)}"
    return r

In [50]:
model = Model(layers = [
    Dense(neurons = 3, activation = sigmoid),
    Dense(neurons = 1)
    ])
model

Layers ................. 
 Dense Layer(neurons = 3) avec Sigmoid 
 Dense Layer(neurons = 1)

In [39]:
Y

array([[ 0.56486772],
       [ 1.48800652],
       [-1.48644887],
       [-0.13822431]])

In [51]:
model.forward(X)

array([[2.47005633],
       [2.53479834],
       [1.89807891],
       [1.92678186]])

In [52]:
loss_derivee = np.random.randn(4,1)
loss_derivee

array([[-0.23415337],
       [-0.23413696],
       [ 1.57921282],
       [ 0.76743473]])

In [53]:
model.backward(loss_derivee)

array([[-0.02166009, -0.00198156,  0.00600408],
       [-0.01558369,  0.00516281,  0.01226937],
       [ 0.00662474,  0.0076687 , -0.01500028],
       [ 0.01131573,  0.00059134, -0.01919523]])

In [54]:
model.get_params()

[[array([[ 0.49671415, -0.1382643 ,  0.64768854],
         [ 1.52302986, -0.23415337, -0.23413696],
         [ 1.57921282,  0.76743473, -0.46947439]]),
  array([[ 0.54256004, -0.46341769, -0.46572975]])],
 [array([[ 0.49671415],
         [-0.1382643 ],
         [ 0.64768854]]),
  array([[1.52302986]])]]

In [55]:
model.get_derivee_params()

[[array([[-0.02242822,  0.11838461, -0.17179535],
         [-0.01212565, -0.11400805, -0.12785598],
         [ 0.01783758, -0.13599864,  0.14880161]]),
  array([[-0.00327309, -0.03532094, -0.03483333]])],
 [array([[ 1.88551641],
         [ 1.99795603],
         [-0.33476421]]),
  array([[1.87835721]])]]

# Update des paramètres

In [43]:
M

Dot Product

In [44]:
M.param

array([[ 0.4967],
       [-0.1383],
       [ 0.6477]])

In [45]:
M.derivee_inputs

array([[ 0.79234626, -0.22061906,  1.03322463],
       [-0.27898026,  0.07767862, -0.36379206],
       [-0.53716203,  0.14956615, -0.70046274],
       [ 0.59720389, -0.16628407,  0.77875772]])

In [46]:
couche.params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [47]:
couche.derivee_params

[array([[-0.30320043,  0.20796531],
        [-0.06424681, -0.07766607],
        [ 0.04724885, -0.1280065 ]]),
 array([[ 0.02131064, -0.03919074]])]

In [48]:
for p in params:
  params[p] -= learning_rate*grads[p]

NameError: ignored